# Building Intelligent Agents with Tools and Memory

This comprehensive tutorial covers building sophisticated AI agents using the Agno framework. You'll learn to create agents with various capabilities: basic conversation, web search, code execution, custom tools, and persistent memory.

**What you'll learn:**
- Agent architecture and tool integration
- Google Search and Python code execution tools
- Custom function creation and integration
- Persistent memory systems with SQLite
- Conversation history and user context management

## Step 1: Environment Setup

Installing the Agno framework - a powerful Python library for building AI agents. Agno provides:
- Pre-built tools for common tasks (search, code execution)
- Memory systems for context retention
- Easy integration with various language models
- Agent orchestration capabilities

In [1]:
!pip install -U -q agno

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.1/948.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 21.6 MB/s eta 0:00:00


## Step 2: Additional Dependencies

Installing python-dotenv for secure API key management. This allows us to store sensitive information like OpenAI API keys in environment files rather than hardcoding them in our notebooks.

In [2]:
!pip install python-dotenv

In [3]:
!pwd

/content


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 3: Google Colab Configuration

Setting up Google Drive access and navigation for Colab environment. This allows us to:
- Access API keys stored in Google Drive
- Maintain persistent files across sessions
- Work with larger datasets stored in Drive

In [5]:
%cd /content/drive/My Drive/

/content/drive/My Drive


## Step 4: API Key Configuration

Loading OpenAI API key from environment file. This secure approach:
- Keeps sensitive keys out of code
- Allows easy key rotation
- Prevents accidental key exposure in version control

Ensure your `.env` file contains: `OPENAI_API_KEY=your_actual_key_here`

In [ ]:
# Load API key from environment file
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(dotenv_path="/content/drive/My Drive/agentic_ai_tutorials/.env")
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

## Step 5: Basic Agent Creation

Creating our first agent with GPT-4o model. This basic agent can:
- Engage in natural conversation
- Use GPT-4o's advanced reasoning capabilities
- Format responses in Markdown for better readability

The agent serves as our foundation before adding tools and memory.

In [ ]:
# Create a basic conversational agent with GPT-4o
from agno.agent import Agent
from agno.models.openai import OpenAIChat

agent = Agent(
    model=OpenAIChat(id="gpt-4o"),  # Using GPT-4o for advanced reasoning
    description="You're a cheerful AI pal who loves a good chat!",
    markdown=True  # Enable Markdown formatting for responses
)

## Step 6: Testing Basic Conversation

Testing our basic agent with a simple query. The streaming response shows how the agent generates text in real-time, providing a better user experience for longer responses.

In [8]:
agent.print_response("Summarize the story of 'The Lion King.", stream=True)

Output()

## Step 7: Adding Web Search Capabilities

Installing dependencies for Google Search integration. Tools extend agent capabilities beyond their training data by allowing them to:
- Access real-time information
- Perform specific actions (search, calculations, API calls)
- Interact with external systems

**Tools** are functions that agents can call when needed to accomplish tasks.

In [10]:
!pip install -U googlesearch-python pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.1 MB/s eta 0:00:00


## Step 8: Creating Search-Enabled Agent

Building an agent with Google Search capabilities. Configuration details:
- **GoogleSearchTools**: Enables web search functionality
- **show_tool_calls=True**: Displays when and how tools are used
- **stop_after_tool_call_tools**: Controls execution flow after search

This agent can now access current information beyond its training cutoff.

In [ ]:
# Create agent with Google Search capabilities
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.googlesearch import GoogleSearchTools

agent = Agent(
    model=OpenAIChat(id="gpt-4o-mini"),  # More cost-effective model for search tasks
    tools=[
        GoogleSearchTools(
            stop_after_tool_call_tools=["google_search"],  # Control execution flow
            show_result_tools=["google_search"],           # Display search results
        )
    ],
    show_tool_calls=True,  # Show when and how tools are used
)

## Step 9: Testing Web Search

Demonstrating how the agent uses search tools automatically. Watch how the agent:
1. Recognizes it needs current information
2. Calls the Google Search tool
3. Processes search results
4. Synthesizes a comprehensive response

In [ ]:
# Test the search-enabled agent with a current events query
agent.print_response("What are the trending AI tools in 2025?", stream=True)

Output()

## Step 10: Python Code Execution Agent

Creating an agent that can write and execute Python code in real-time. **PythonTools** enable the agent to:
- Write Python functions and scripts
- Execute code safely in a sandboxed environment
- Debug and iterate on solutions
- Perform calculations, data analysis, and more

This transforms the agent into a programming assistant.

In [ ]:
# Create agent with Python code execution capabilities
from agno.agent import Agent
from agno.tools.python import PythonTools

agent = Agent(
    tools=[PythonTools()],    # Enable Python code execution
    show_tool_calls=True      # Display code execution process
)

## Step 11: Testing Code Execution

Asking the agent to solve a programming problem. Observe how it:
1. Understands the requirement
2. Writes Python code
3. Executes and tests the solution
4. Explains the approach

In [15]:
agent.print_response("Write a function to reverse a string without using slicing")

INFO Setting default model to OpenAI Chat

Output()

WARNING  PythonTools can run arbitrary code, please provide human supervision.

INFO Saved: /content/drive/MyDrive/reverse_string.py

INFO Running /content/drive/MyDrive/reverse_string.py

INFO Saved: /content/drive/MyDrive/reverse_string.py

INFO Running /content/drive/MyDrive/reverse_string.py

## Step 12: Creating Custom Tools

Building custom functions that agents can use. **Custom tools** allow you to:
- Extend agent capabilities with domain-specific functions
- Integrate with APIs and external services
- Create reusable components for your agents

The function docstring helps the agent understand when and how to use the tool.

In [ ]:
# Create a custom tool function
from datetime import datetime
from agno.agent import Agent

def get_today_date() -> str:
    """
    Returns today's date in a formatted string.
    This tool provides current date information to the agent.
    """
    today = datetime.now().strftime("Today is %B %d, %Y")
    return today

# Test the function
get_today_date()

'Today is July 22, 2025'

## Step 13: Integrating Custom Tools

Adding our custom function to an agent. The agent automatically:
- Recognizes when to use the tool based on user queries
- Calls the function when date information is needed
- Incorporates the result into its response

This demonstrates how easily custom functionality can be added to agents.

In [ ]:
# Create agent with custom tool
agent = Agent(
    tools=[get_today_date],    # Add our custom function as a tool
    show_tool_calls=True,      # Show when the tool is called
    markdown=True              # Format responses in Markdown
)

# Test the agent - it will automatically use the tool when needed
agent.print_response("What is today's date?", stream=True)

INFO Setting default model to OpenAI Chat

Output()

## Step 14: Advanced Custom Tool - Wikipedia Search

Creating a more sophisticated tool that makes HTTP requests to the Wikipedia API. This demonstrates:
- **API Integration**: Connecting to external services
- **Error Handling**: Managing failed requests gracefully
- **Data Processing**: Extracting useful information from API responses

This tool enables the agent to access real-time Wikipedia content.

In [ ]:
# Advanced custom tool: Wikipedia search with API integration
import httpx
from agno.agent import Agent

def search_wikipedia(topic: str = "Machine learning") -> str:
    """
    Searches Wikipedia for a given topic and returns a summary.
    
    Args:
        topic: The topic to search for on Wikipedia
    
    Returns:
        Formatted summary with title and extract
    """
    # Construct Wikipedia API URL with URL encoding
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic.replace(' ', '%20')}"
    
    try:
        # Make HTTP request with timeout for reliability
        response = httpx.get(url, timeout=5.0)
        data = response.json()
        
        # Extract summary information from API response
        if "extract" in data:
            title = data.get("title", topic)
            extract = data["extract"]
            return f"**{title}**:\n{extract}"
        else:
            return "Sorry, I couldn't find anything on that topic."
    except Exception as e:
        return f"Error searching Wikipedia: {str(e)}"

## Step 15: Testing Wikipedia Tool

Testing our Wikipedia search tool with an agent. The agent will automatically determine when to use Wikipedia based on the user's query and call the appropriate function to retrieve information.

In [19]:
agent = Agent(
    tools=[search_wikipedia],
    show_tool_calls=True,
    markdown=True
)

# Example prompt
agent.print_response("Explain quantum computing in simple words", stream=True)

INFO Setting default model to OpenAI Chat

Output()

## Step 16: Memory-Enabled Agents

Building agents with persistent memory capabilities. **Memory systems** enable agents to:
- Remember previous conversations across sessions
- Build context about users over time
- Provide personalized responses based on history
- Maintain state between interactions

This transforms agents from stateless responders to context-aware assistants.

In [ ]:
# Create agent with comprehensive memory system
from agno.agent import Agent
from agno.memory.v2.memory import Memory
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.storage.sqlite import SqliteStorage
from rich.pretty import pprint

# Configuration
db_file = "tmp/agent_storage.db"  # SQLite database file for persistence
user_id = "Aarohi"                # Unique identifier for user

# Initialize memory system with SQLite backend
memory = Memory(
    db=SqliteMemoryDb(table_name="user_memories", db_file=db_file),
)

# Create agent with memory and storage capabilities
agent = Agent(
    model=OpenAIChat(id="gpt-4o"),           # Advanced model for memory processing
    description="You're an AI with a memory!",
    memory=memory,                            # Attach memory system
    storage=SqliteStorage(                    # Persistent conversation storage
        table_name="agent_sessions", 
        db_file=db_file
    ),
    enable_user_memories=True,                # Enable automatic memory creation
    add_history_to_messages=True,             # Include conversation history
    num_history_runs=3,                       # Number of previous runs to include
    session_id="my_chat_session",             # Session identifier
    markdown=True,                            # Enable Markdown formatting
)

## Step 17: Testing Memory Formation

Testing how the agent creates and stores memories. The agent will:
1. Process the user's message
2. Extract important information to remember
3. Store memories with relevant topics
4. Use this context in future conversations

In [22]:
agent.print_response("I love South Indian food. What’s your favorite cuisine?", user_id=user_id, stream=True)


Output()

## Step 18: Inspecting Stored Memories

Examining what the agent remembered from our conversation. This shows how the memory system:
- Extracts key information from conversations
- Associates memories with relevant topics
- Organizes information for future retrieval

In [23]:
print("\n Current **memories** about the user:")
user_memories = memory.get_user_memories(user_id=user_id)
pprint([{"memory": m.memory, "topics": m.topics} for m in user_memories])



 Current **memories** about the user:


[{'memory': 'User loves South Indian food.', 'topics': ['food', 'preferences']}]

## Step 19: Testing Memory Recall

Testing how the agent uses stored memories to provide contextual responses. The agent should recall previous information about the user and incorporate it into new conversations.

In [24]:
agent.print_response("What did I just say I love and also I love hill stations", user_id=user_id)



Output()

## Step 20: Final Memory State

Reviewing the complete memory profile built for the user. This demonstrates how persistent memory enables:
- Cumulative learning about users
- Personalized interactions
- Context preservation across sessions
- Building long-term relationships with users

In [ ]:
# Final inspection of all stored memories
print("\n Current **memories** about the user:")
user_memories = memory.get_user_memories(user_id=user_id)
pprint([{"memory": m.memory, "topics": m.topics} for m in user_memories])


 Current **memories** about the user:


[
│   {'memory': 'User loves hill stations.', 'topics': ['preferences']},
│   {'memory': 'User loves South Indian food and hill stations.', 'topics': ['preferences']}
]

## Conclusion

You've successfully built intelligent agents with various capabilities:

**Key Concepts Learned:**
- **Basic Agents**: Foundation for AI interactions
- **Tools**: Extend agent capabilities (search, code execution, custom functions)
- **Memory Systems**: Enable persistent context and user personalization
- **Agent Architecture**: How components work together

**Next Steps:**
- Experiment with different tool combinations
- Build domain-specific custom tools
- Implement more sophisticated memory patterns
- Integrate with external APIs and services

The Agno framework provides a solid foundation for building production-ready AI agents with advanced capabilities.